### SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

![](db.png)

### Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
def select(sql):
    result = pd.io.sql.read_sql(query, con = engine)
    return result

#### Посчитайте, сколько книг вышло после 1 января 2000 года;

In [4]:
query = ''' SELECT COUNT(title)
        FROM books
        WHERE publication_date > '2000-01-01'
        '''

In [5]:
print(f'После 1 января 2000 года вышло: {select(query).iat[0,0]} книг')

После 1 января 2000 года вышло: 819 книг


#### Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [6]:
query = ''' SELECT b.title,
            b.book_id,
            AVG(ra.rating) as average_rating,
            COUNT(DISTINCT r.text) as number_of_reviews
            FROM books as b
            LEFT JOIN ratings as ra ON b.book_id = ra.book_id
            LEFT JOIN reviews as r ON b.book_id = r.book_id

            GROUP BY b.title,
            b.book_id

            ORDER BY number_of_reviews DESC
        '''

In [7]:
select(query)

,title,book_id,average_rating,number_of_reviews
0,Twilight (Twilight #1),948,3.662500,7
1,Water for Elephants,963,3.977273,6
2,The Glass Castle,734,4.206897,6
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,4.414634,6
4,The Curious Incident of the Dog in the Night-Time,695,4.081081,6
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,3.666667,0
996,The Natural Way to Draw,808,3.000000,0
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,5.000000,0
998,Essential Tales and Poems,221,4.000000,0


#### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [8]:
query = ''' SELECT p.publisher, COUNT(title) as count_books
        FROM books as b
        INNER JOIN publishers AS p ON b.publisher_id = p.publisher_id
        WHERE b.num_pages > 50
        GROUP BY p.publisher
        ORDER BY count_books DESC
        LIMIT 1
        '''

In [9]:
print(f'Издательство {select(query).iat[0,0]} выпустило наибольшее число ({select(query).iat[0,1]} книги), толще 50 страниц')

Издательство Penguin Books выпустило наибольшее число (42 книги), толще 50 страниц


#### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [10]:
query = '''
        WITH
        r_50 as (
        SELECT b.book_id, b.title, COUNT(r.rating_id) as count_rating
        FROM books as b
        INNER JOIN authors AS a ON b.author_id = a.author_id
        INNER JOIN ratings AS r ON b.book_id = r.book_id
        GROUP BY b.book_id, b.title
        HAVING COUNT(r.rating_id) >= 50)
        
        SELECT a.author, ROUND(AVG(ra.rating),2) as mean_rating
        FROM r_50 as r
        INNER JOIN books as b on r.book_id = b.book_id
        INNER JOIN authors as a on b.author_id = a.author_id
        INNER JOIN ratings as ra on b.book_id = ra.book_id
        GROUP BY a.author
        ORDER BY mean_rating DESC
        LIMIT 1
        '''

In [11]:
print(f'Автор {select(query).iat[0,0]} имеет самый высокий рейтинг оценки его книг ({select(query).iat[0,1]} балла)')

Автор J.K. Rowling/Mary GrandPré имеет самый высокий рейтинг оценки его книг (4.29 балла)


#### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
query = '''
        WITH 
        users_50 as(
        SELECT username, COUNT(rating_id) as count_rating
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating_id) > 50)
        
        SELECT ROUND(AVG(count_review),2)
        FROM(SELECT username, COUNT(text) AS count_review
        FROM reviews
        WHERE username IN (SELECT username
                            FROM users_50)
        GROUP BY username) AS kek
        
        '''

In [13]:
print(f'Среднее кол-во обзоров от пользователей, которые поставили больше 50 оценок составляет {select(query).iat[0,0]} штук')

Среднее кол-во обзоров от пользователей, которые поставили больше 50 оценок составляет 24.33 штук


<strong> Выводы:</strong>

- После 1 января 2000 года вышло: 819 книг;
- Издательство Penguin Books выпустило наибольшее число (42 книги), толще 50 страниц;
- Автор J.K. Rowling/Mary GrandPré имеет самый высокий рейтинг оценки его книг (4.29 балла);
- Среднее кол-во обзоров от пользователей, которые поставили больше 50 оценок составляет (24.33 штук);